SI 671 Project 

In [2]:
import pandas as pd
import numpy as np 
import json
business_path = "Data/yelp_academic_dataset_business.json"
checkin_path = "Data/yelp_academic_dataset_checkin.json"
review_path = "Data/yelp_academic_dataset_review.json"
tip_path = 'Data/yelp_academic_dataset_tip.json'
user_path = 'Data/yelp_academic_dataset_user.json'

# load data

In [5]:
def json_to_df(path):
    with open(path) as data_file:
        data = []
        for line in data_file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    return df

In [7]:
df_business = json_to_df(business_path)
df_checkin = json_to_df(checkin_path)
df_review = json_to_df(review_path)
df_tip = json_to_df(tip_path)
df_user = json_to_df(user_path)